In [2]:
import pandas as pd
from geograpy import places 
import re 
import geopy
from geopy.geocoders import ArcGIS 
import numpy as np

import requests
from bs4 import BeautifulSoup

import json
from json import loads, dumps


In [3]:
data = pd.read_excel("SanAntonioConcerts.xlsx", sheet_name='Year')

data

,Month,Day,Year,Artist,Venue,Unnamed: 5,Notes,Unnamed: 7
0,June,25,1869.0,General Mason Farewell Party: San Antonio Bras...,NaN,NaN,SAE 06/27/1869 p. 3,NaN
1,July,4,1874.0,San Antonio Brass Band,San Pedro Park,NaN,SAE 07/04/1874 Ad p. 2,NaN
2,October,9,1874.0,Tenth Texas Saengerfest,"San Pedro Springs, Turner Hall",NaN,SAE 10/9/1874 p. 3,NaN
3,October,10,1874.0,Tenth Texas Saengerfest,"Casino Hall, Turner Hall",NaN,SAE 10/9/1874 p. 3,NaN
4,October,11,1874.0,Tenth Texas Saengerfest,"San Pedro Springs, Alamo Plaza, Casino Hall",NaN,SAE 10/9/1874 p. 3,NaN
...,...,...,...,...,...,...,...,...
20099,NaN,NaN,NaN,(03/21/2015) opening of Paper Tiger,NaN,NaN,NaN,NaN
20100,NaN,NaN,NaN,(09/16/2022) San Antonio Philharmonic Orchestr...,NaN,NaN,NaN,NaN
20101,NaN,NaN,NaN,(03/04/2023) Reopening of The Espee (Sunset St...,NaN,NaN,NaN,NaN
20102,NaN,NaN,NaN,^ Willie Nelson's first performance at Gruene ...,NaN,NaN,NaN,NaN


In [4]:
data.dtypes

Month          object
Day            object
Year          float64
Artist         object
Venue          object
Unnamed: 5    float64
Notes          object
Unnamed: 7     object
dtype: object

In [9]:
# adding address column 
data['Address'] = None

# Filter the DataFrame based on a specific value range from 'col1' and select 'col2' and 'col3'
filtered_data = data[(data['Year'] >= 1970) & (data['Year'] <= 2010)][['Year', 'Address','Venue', "Artist"]]

filtered_data

,Year,Address,Venue,Artist
2404,1970.0,None,Hemisfair Arena,"KBER Holiday Show: Willie Nelson, Charley Pri..."
2405,1970.0,None,Municipal Auditorium,Vikki Carr
2406,1970.0,None,Hemisfair Arena,"Chicago Transit Authority, Youngbloods"
2407,1970.0,None,Joe Freeman Coliseum,James Brown
2408,1970.0,None,Randy's Rodeo,Woody Herman and his 16 piece orchestra
...,...,...,...,...
13739,2010.0,None,Tra's Country,Cadilac Ranch
13740,2010.0,None,Tra's Country,Cadilac Ranch
13741,2010.0,None,Quihi Gun Club & Dance Hall,28th Christmas Dance: Cactus Country
13742,2010.0,None,Christ Church,Baroque Ensemble Retablo


In [10]:
filtered_data['Year'] = (filtered_data['Year']*1).astype(int)

filtered_data.dtypes

Year        int32
Address    object
Venue      object
Artist     object
dtype: object

In [11]:
og_data = filtered_data.to_json(orient='records', indent=4)
#print(json_df)
file_path = 'D:\MusicPY\og_data.json'
with open(file_path, 'w') as f:
    f.write(og_data)

print(og_data)

[
    {
        "Year":1970,
        "Address":null,
        "Venue":"Hemisfair Arena",
        "Artist":"KBER Holiday Show:  Willie Nelson, Charley Pride, Grandpa Jones, David Houston"
    },
    {
        "Year":1970,
        "Address":null,
        "Venue":"Municipal Auditorium",
        "Artist":"Vikki Carr"
    },
    {
        "Year":1970,
        "Address":null,
        "Venue":"Hemisfair Arena",
        "Artist":"Chicago Transit Authority, Youngbloods"
    },
    {
        "Year":1970,
        "Address":null,
        "Venue":"Joe Freeman Coliseum",
        "Artist":"James Brown"
    },
    {
        "Year":1970,
        "Address":null,
        "Venue":"Randy's Rodeo",
        "Artist":"Woody Herman and his 16 piece orchestra"
    },
    {
        "Year":1970,
        "Address":null,
        "Venue":"Farmer's Daughter",
        "Artist":"Bob Wills Benefit Concert: Johnny Bush and the Bandoleros, Leon Rausch, Bobby Stone and the\nNewsboys, Rusty McDonald, Slim Roberts and His Men

In [8]:
satx_venues_path = 'D:\MusicPY\TexasMusicVenue_SanAntonio.json'
with open(satx_venues_path, 'r') as f:
    satx_venue_data = json.load(f)

print(json.dumps(satx_venue_data, indent=4))

[
    {
        "Year": null,
        "Address": "1 AT&T Center, San Antonio, TX 78219",
        "Venue": "AT&T Center"
    },
    {
        "Year": null,
        "Address": "100 Montana Street, San Antonio, TX 78205",
        "Venue": "Alamodome"
    },
    {
        "Year": null,
        "Address": "8736 Tesoro Drive, San Antonio, TX 78217",
        "Venue": "Arjon's"
    },
    {
        "Year": null,
        "Address": "418 Villita Street, San Antonio, TX 78205",
        "Venue": "Arneson River Theatre"
    },
    {
        "Year": null,
        "Address": "104 North Saint Mary's Street, San Antonio, TX 78205",
        "Venue": "Aztec Theatre | San Antonio"
    },
    {
        "Year": null,
        "Address": "713 South Alamo, San Antonio, TX 78205-3403",
        "Venue": "Azuca Cocina Latina Nuevo"
    },
    {
        "Year": null,
        "Address": "119 El Mio Street, San Antonio, TX 78216",
        "Venue": "The Bang Bang Bar"
    },
    {
        "Year": null,
        "Addre